# Gen AI Population Simulation Testing for AI Workflow Automation Validation

## Goal
The goal is to generate a population of banking users and use them to simulate banking operations, to test AI workflow automations. 

The idea here is to perform a population testing on banking systems using AI, especially AI automated workflows

## Strategy

The strategy here is to generate customer profiles using Gen AI and use Gen AI is create backstories for these customers. Once we have customers with backstory we simulate an event in customer life which requires interaction with their banks or wealth advisor. This interaction we be handled by an automated AI agent and we see how good the system is able to handle this scenario

## Implementation
1. Define the customer schema and the properties the customer should have.
2. We create a agentic flo which will generate such customers randomly, the agent saves this data to  SQLite database
3. The flo also has an agent which can query the created database which will help the agentic flo to check back the distribution of generated customers
4. Once we have the generated customers, we loop through each customer and use their backstore to create a life event, requiring banking help. This will be a new agentic flo which can trigger different banking apis like customer support or sending email etc.

Through this excersice we create AI generated customer population and through this we test our automations

## 1. Define the customer schema
We will define an elaborate customer schema to have all the variables associated with the customer

In [41]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import List
from peewee import Model, SqliteDatabase, CharField, IntegerField, TextField
from dotenv import load_dotenv

load_dotenv()

class Customer(BaseModel):
    # base info
    name: str = Field("Name of the customer")
    age: int = Field("Age of the customer")
    income: int = Field("Annual income in INR")

    #assets
    cash: int = Field("Total cash in hand in INR")
    equities: int = Field("Amount invested in equities market in INR")
    realEstate: int = Field("Amount invested in real estate in INR")
    retirementAccounts: int = Field("Amount invested in pension schemes in INR")
    commodities: int = Field("Amount invested in gold or silver in INR")
    alternativeInvestments: int = Field("Amount invested in private equity and hedge funds in INR")
    cars: int = Field("Market value of cars owned in INR")
    bikes: int = Field("Market value of bikes owned in INR")
    insurance: int = Field("Coverage across health and life insurance in INR")

    # debts
    studentLoans: int = Field("Amount of student debt in INR")
    creditCardDebt: int = Field("Amount of credit card debt in INR")
    autoLoans: int = Field("Amount of automobile loan in INR")
    personalLoans: int = Field("Amount of personal loan in INR")
    homeMortgages: int = Field("Amount of home loan in INR")
    smallBusinessLoans: int = Field("Amount of business loans in INR")
    medicalDebt: int = Field("Amount of medical loan in INR")

    # salary status
    salaryStatus: str = Field("""Salary status of the customer, it should one of these:
                                "salaried",
                                "unemployed",
                                "selfEmployed",
                                "contract",
                                "intern",
                                "retired"
                              """)

    # financial goals
    financical_goal: str = Field("""
        A comma seperated list of financial goals the customer would like to persue in the future.
        Try to make it consistent with customers backstory. Following are the possible values:
                            
        "educationFunding"
        "startingInvestments"
        "smallPurchases"
        "debtRepayment"
        "buildingEmergencyFund"
        "firstCarPurchase"
        "homeDownPayment"
        "wealthBuilding"
        "insurance"
        "retirementPlanning"
        "childrenEducation"
        "homePurchase"
        "investmentDiversification"
        "taxPlanning"
        "emergencyFundExpansion"
        "retirementFundGrowth"
        "homeRenovation"
        "longTermInvestments"
        "childrenHigherEducation"
        "healthAndLifeInsurance"
        "philanthropy"
        "retirementIncomeManagement"
        "travelAndLeisure"
        "sustainableIncome"
        "lifestyleMaintenance"
        "legacyBuilding"
        "giftingAndDonations"
        "assistedLiving"
        "wealthDistribution"
    """)

    # life events
    life_events: str = Field("""
        A comma seperated list of life events that happened in life
        Try to make it consistent with customers backstory. Following are the possible values:

        "graduation"
        "higherEducationEnrollment"
        "firstJob"
        "jobPromotions"
        "firstVehiclePurchase"
        "homePurchase"
        "marriage"
        "jointFinancialPlanning"
        "childbirth"
        "jobLoss"
        "careerTransition"
        "entrepreneurship"
        "seriousIllness"
        "disability"
        "receivingAnInheritance"
        "divorce"
        "retirement"
        "widowhood"
        "largeUnexpectedExpenses"
        "homeRepairs"
        "legalIssues"
        "relocation"
        "movingToANewCityCountry"
        "buyingASecondHome"
        "philanthropy"
        "largeDonations"
    """)


    backstory: str = Field(""" 
        A backstory of the user in less than 300 words, about the persons career, education and up and downs in their life.
        Be creative in building the backstory and try to make it different for everyone
    """)

## 2. Define the SQL Insert Tool

Next we create a tool which can do the following things:

1. Take in customer list as input
2. Create an SQLite db, and a customer table if they dont exist already
3. Insert the customers into the table

This tool will be used by our agent to store customer data

In [42]:
class SQLLiteInsertInput(BaseModel):
    customers: List[Customer] = Field(description="The list of banking customers to be saved")

class SQLLiteInsertTool(BaseTool):
    name = "sql_insert_tool"
    description = "useful for inserting customer profile to sqllite db"
    args_schema: Type[BaseModel] = SQLLiteInsertInput

    def _run(
        self, customers: List[Customer]
    ) -> str:
        try:
            db = SqliteDatabase('population.sql')
            class Customers(Model):
                name = CharField(max_length=255)
                age = IntegerField()
                income = IntegerField()

                # assets
                cash = IntegerField()
                equities = IntegerField()
                realEstate = IntegerField()
                retirementAccounts = IntegerField()
                commodities = IntegerField()
                alternativeInvestments = IntegerField()
                cars = IntegerField()
                bikes = IntegerField()
                insurance = IntegerField()

                # debts
                studentLoans = IntegerField()
                creditCardDebt = IntegerField()
                autoLoans = IntegerField()
                personalLoans = IntegerField()
                homeMortgages = IntegerField()
                smallBusinessLoans = IntegerField()
                medicalDebt = IntegerField()

                # salary status
                salaryStatus = CharField(max_length=255)

                # financial goals
                financical_goal = TextField()

                # life events
                life_events = TextField()

                # backstory
                backstory = TextField()

                class Meta:
                    database = db

            db.connect()
            db.create_tables([Customers])

            with db.atomic():
                [Customers.create(**x.dict()) for x in customers]
            
        except Exception as error:
            print (f"Failed to insert data into sqlite: {error}")
            return f"Insert failed with error, {error}"
        finally:
            db.close()
            print("The SQLite connection is closed")
            return "Insert sucess"
        
    async def _arun(
        self, email: str
    ) -> str:
        raise Exception("Not implemented")

## 3. Create SQLite query tool

Next we create an SQLite Query tool which does the follows:

1. Take query string as input
2. Connect to the customers table and run the query and return the result.

In [43]:
class SQLLiteQueryInput(BaseModel):
    query: str = Field(description="Valid SQLite query for fetching customer information")

class SQLLiteQueryTool(BaseTool):
    name = "sql_query_tool"
    description = "useful for querying customer data from the sqlite db"
    args_schema: Type[BaseModel] = SQLLiteQueryInput

    def _run(
        self, query: str
    ) -> str:
        db = SqliteDatabase('population.sql')
        db.connect()

        results = db.execute_sql(query).fetchall()
        result_str = "Query Results:\n"
        for row in results:
            result_str += f" - {row}\n"
        return result_str

    async def _arun(
        self, email: str
    ) -> str:
        raise Exception("Not implemented") 

## 4. Set up the LLM
Ready the LLM to run the agent flo with.
You are free to change this to your open ai backend by changing this to ChatOpenAI

In [44]:
from langchain_openai import AzureChatOpenAI
from flo_ai import FloSession
import os

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_GPT4_ENDPOINT"),
    model_name="gpt-4",
    temperature=0.9,
    max_tokens=4096,
    api_version="2023-03-15-preview",
    api_key=os.getenv("AZURE_OPEN_AI_API_KEY")
)

## 5. Setup customer population generator agent

This will be a agentic team. The team will have 2 agents:

Agent 1: An agent to generate the list of customers and with the ability to save the sqllite

Agent 2: An agent that can query the sqlite db to understand the customer demographics and distribution

We use these two agents to work together to generate desired number of customers

In [45]:
agent_yaml = """
apiVersion: flo/alpha-v1
kind: FloRoutedTeam
name: population-generator
team:
    name: PopulationGeneratorTeam
    router:
        name: PopulatioGenerationSupervisor
        kind: supervisor
    agents:
      - name: CustomerGenerator
        job: You job is to generate a list of bank customers with name and age, then save it to sqlite db.
        tools:
          - name: SQLLiteInsertTool
      - name: CustomerDataFetcher
        job: >
            Your job is to build and run queries to answer the questions asked.

            Here is the customer peewee model, the name of the table is "customers":

            name = CharField(max_length=255)
            age = IntegerField()
            income = IntegerField()
            cash = IntegerField()
            equities = IntegerField()
            realEstate = IntegerField()
            retirementAccounts = IntegerField()
            commodities = IntegerField()
            alternativeInvestments = IntegerField()
            cars = IntegerField()
            bikes = IntegerField()
            insurance = IntegerField()
            studentLoans = IntegerField()
            creditCardDebt = IntegerField()
            autoLoans = IntegerField()
            personalLoans = IntegerField()
            homeMortgages = IntegerField()
            smallBusinessLoans = IntegerField()
            medicalDebt = IntegerField()
            salaryStatus = CharField(max_length=255)
            financical_goal = TextField()
            life_events = TextField()
            backstory = TextField()
            
        tools:
          - name: SQLLiteQueryTool
"""

session = FloSession(llm)

session.register_tool(
    name="SQLLiteInsertTool", 
    tool=SQLLiteInsertTool()
).register_tool(
    name="SQLLiteQueryTool", 
    tool=SQLLiteQueryTool()
)

## 6. Build and run

Use the flo builder to build and run the flo to generate the customers.

In [46]:
from flo_ai import Flo

flo: Flo = Flo.build(session, yaml=agent_yaml)
for s in flo.stream("""
Create 10 banking customer personas whose demographics & financial profiles mimic the distribution of banking customers 
in a large bank in India, and insert them to db. Do note that these customer personas should represent all income brackets, employment statuses. 
After creating the first set of 10 personas, create another 10 profiles so that personas mimic a customer distribution even more accurately and insert it
You can choose to create and insert as batch if that is best for you
"""):
     if "__end__" not in s:
        print(s)
        print("----")

{'PopulatioGenerationSupervisor-t6t7S': {'next': 'CustomerGenerator-3lHy7'}}
----


> Entering new AgentExecutor chain...
I will start by creating the first batch of 10 customer personas representing a diverse range of demographics and financial profiles akin to a large bank's clientele in India. These will include various income brackets and employment statuses. After creating and reviewing this first batch, I'll proceed with the second set of 10 customer personas. Let's start with the first 10:

### First Batch of Customer Personas:

1. **Name**: Amit Sharma, **Age**: 35
   - **Income**: 1200000 (Annual)
   - **Financial Profile**:
     - Cash: 50000
     - Equities: 200000
     - Real Estate: 5000000
     - Retirement Accounts: 300000
     - Commodities: 10000
     - Alternative Investments: 50000
     - Cars: 600000
     - Bikes: 150000
     - Insurance: 100000
     - Student Loans: 0
     - Credit Card Debt: 10000
     - Auto Loans: 200000
     - Personal Loans: 0
     - Home Mort

# Execution Phase

This is the next phase we use these customers to generate lifevents which needs banking help and create an agent to simulate the same

We are gonna create 3 functionalities the bank supports:

1. Purchase items: Where the customer can purchase items from outside and the customer account balance comes down
2. Send a customer support email
3. Request for a loan

In [47]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class Purchase(BaseModel):
    purchase_item: str = Field("The item purchase like TV, microwave, laptop etc")
    purchase_price: int = Field(description="A payment amount made by the customer")

class PurchaseTool(BaseTool):
    name = "transaction"
    description = "useful for when you want to by something"
    args_schema: Type[BaseModel] = Purchase

    def _run(
        self, purchase_item: str, purchase_price: int
    ) -> str:
        print(f"{purchase_item} @ {purchase_price}")
        return "Completed transaction successfully"

    async def _arun(
        self, purchase_item: str, purchase_price: int
    ) -> str:
        print(f"{purchase_item} @ {purchase_price}")
        return "Completed transaction successfully"

In [48]:
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class CustomerSupportInput(BaseModel):
    email: str = Field(description="The email text to be sent to customer support")

class CustomerSupportTool(BaseTool):
    name = "customer_support_tool"
    description = "useful for when you need to send an email to customer support"
    args_schema: Type[BaseModel] = CustomerSupportInput

    def _run(
        self, email: str
    ) -> str:
        print(email)
        return "Email sent successfully"

    async def _arun(
        self, email: str
    ) -> str:
        print(email)
        return "Email sent successfully"

In [49]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class LoanRequest(BaseModel):
    request: str = Field(description="An message asking for a loan along with loan amount, loan type and reason for the loan")

class LoanRequestTool(BaseTool):
    name = "loan_request"
    description = "useful for when you need a loan from the bank"
    args_schema: Type[BaseModel] = LoanRequest

    def _run(
        self, request: str
    ) -> str:
        print(request)
        return "Loan request successful"

    async def _arun(
        self, request: str
    ) -> str:
        print(request)
        return "Loan request successful"

## Set up the customer simulator agent

This is the agent which will mock customer life event

In [54]:
execution_yaml = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: banking-assistant
agent:
    name: BankingCustomer
    job: >
      You have the capability to interact with the bank in different ways. Depending upon your need take the right actions
    tools:
      - name: PurchaseTool
      - name: LoanRequestTool
      - name: CustomerSupportTool
"""

from flo_ai import FloSession

session = FloSession(llm)

session.register_tool(
    name="PurchaseTool", 
    tool=PurchaseTool()
).register_tool(
    name="LoanRequestTool", 
    tool=LoanRequestTool()
).register_tool(
    name="CustomerSupportTool",
    tool=CustomerSupportTool()
)

## Simulation

We read each customer from the SQLite db and use the simulator to mock an event, which will in turn interact with a banking backend

In [58]:
from peewee import SqliteDatabase

db = SqliteDatabase('population.sql')
db.connect()

results = db.execute_sql("SELECT * from customers").fetchall()

customer = results[3]

pr = f"""
Based on the customers backstory & financial goals, create a scenario where user might need a banking help, and take the required action

Here are the customer details:
{customer}
"""

from flo_ai import Flo

flo: Flo = Flo.build(session, yaml=execution_yaml)
for s in flo.stream(pr):
     if "__end__" not in s:
        print(s)
        print("----")


db.close()



> Entering new AgentExecutor chain...
{'actions': [ToolAgentAction(tool='loan_request', tool_input={'request': 'I am Sunita Iyer, a freelance graphic designer, reaching out to request financial assistance to help stabilize my financial situation and support my upcoming goals. I am currently looking to consolidate my debts totaling INR 50,000 to streamline my finances and ease the burden of irregular income flows. Additionally, as I plan to start a family soon, I seek additional financial support to cover potential expenses related to this significant life event. A structured loan plan would greatly assist in managing my finances more efficiently and achieving my personal goals. I appreciate your consideration and look forward to a favorable response.'}, log="\nInvoking: `loan_request` with `{'request': 'I am Sunita Iyer, a freelance graphic designer, reaching out to request financial assistance to help stabilize my financial situation and support my upcoming goals. I am currently loo

True

# Summary

As you can use we generated a population of banking customers and used their profile to generate a backstory. Based on backstory and their financial goals as well as part life events, we created new life events which triggered banking workflows. 

This demonstrate the use of agentic AI for population testing of AI systems.

### How can this be improved ?

1. Add more checks to make sure that the distribution of generatec customers is consistent, meaning its in normal distribution
2. Add a reflection layer in the generator agent team to make sure that enough records are generated